In [1]:
#Log in to Hugging Face Hub (Run BEFORE loading models)
from huggingface_hub import notebook_login

print("Logging into Hugging Face Hub...")
# This will prompt you to enter your access token (the one you just generated)
# Paste the token into the input box that appears below the cell and press Enter.
notebook_login()
print("Hugging Face login process initiated/completed.")


Logging into Hugging Face Hub...


Hugging Face login process initiated/completed.


In [2]:
import wandb

print("Attempting to log in to WandB...")
# This will prompt for your API key interactively in the cell output.
# Find your key at: https://wandb.ai/settings
# Paste the key into the input box below and press Enter.
wandb.login()
print("WandB login process initiated.")


Attempting to log in to WandB...


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kasmik-backups (kasmik-backups-kathmandu-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


WandB login process initiated.


In [3]:
!pip install torch transformer_lens einops datasets huggingface_hub wandb tqdm numpy
import torch
from datasets import load_dataset
import os
from pathlib import Path
import argparse
import json
from IPython import get_ipython # Only needed for arg_parse_update_cfg if running interactively

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 130.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 114.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.2/189.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 39.7 MB/s eta 0:00

In [4]:
def load_pile_lmsys_mixed_tokens(cache_dir="/workspace/cache/", data_dir="/workspace/data/"):
    """ Docstring explaining function purpose, args, and return value """
    # --- Block 1: Setup Paths and Ensure Directories ---
    processed_data_dir = Path(data_dir)
    processed_data_dir.mkdir(parents=True, exist_ok=True)
    token_file_path = processed_data_dir / "pile-lmsys-mix-1m-tokenized-gemma-2.pt"
    # --- Block 2: Attempt to Load from Local Cache ---
    try:
        print(f"Attempting to load processed tokens from: {token_file_path}")
        all_tokens = torch.load(token_file_path)
        print("Loaded processed tokens from disk.")
    # --- Block 3: Handle Cache Miss - Download and Process ---
    except FileNotFoundError:
        print(f"Processed token file not found. Loading dataset from Hugging Face Hub...")
        # Ensure HF cache dir exists
        hf_cache_dir = Path(cache_dir)
        hf_cache_dir.mkdir(parents=True, exist_ok=True)
        # Load from HF
        data = load_dataset(
            "ckkissane/pile-lmsys-mix-1m-tokenized-gemma-2",
            split="train",
            cache_dir=str(hf_cache_dir)
        )
        print("Dataset loaded from Hugging Face Hub.")
        # Format and extract tokens
        data.set_format(type="torch", columns=["input_ids"])
        all_tokens = data["input_ids"]
        # --- Block 4: Save Processed Data to Local Cache ---
        print(f"Saving processed tokens to: {token_file_path}")
        torch.save(all_tokens, token_file_path)
        print(f"Saved tokens to disk.")
    # --- Block 5: Return Tokens ---
    return all_tokens


In [5]:
def arg_parse_update_cfg(default_cfg):
    """ Docstring explaining function purpose """
    # --- Block 1: Check for IPython Environment ---
    ipython = get_ipython()
    if ipython is not None and 'IPKernelApp' in ipython.config:
        print("Running in IPython environment - skipping command line argument parsing.")
        return default_cfg
    # --- Block 2: Initialize Argparse ---
    cfg = dict(default_cfg) # Create a copy
    parser = argparse.ArgumentParser()
    # --- Block 3: Define Arguments Based on Defaults ---
    for key, value in default_cfg.items():
        if type(value) == bool:
            # Special handling for boolean flags
            if value: # Default True -> flag makes it False
                parser.add_argument(f"--{key}", action="store_false", dest=key)
            else:     # Default False -> flag makes it True
                parser.add_argument(f"--{key}", action="store_true", dest=key)
            parser.set_defaults(**{key: value})
        else:
            # Standard argument handling
            parser.add_argument(f"--{key}", type=type(value), default=value)
    # --- Block 4: Parse Arguments and Update Config ---
    args = parser.parse_args()
    parsed_args = vars(args)
    cfg.update(parsed_args)
    # --- Block 5: Print Feedback and Return Config ---
    if cfg != default_cfg:
        print("Updated config based on command line arguments:")
        print(json.dumps(cfg, indent=2))
    else:
        print("Using default configuration.")
    return cfg


CrossCoder Class

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import einops
import json
from pathlib import Path
from typing import NamedTuple, Optional, Union
from huggingface_hub import hf_hub_download
import pprint # For printing config nicely




In [7]:
# Define available data types
DTYPES = {"fp32": torch.float32, "fp16": torch.float16, "bf16": torch.bfloat16}


# Define a structure for the loss output for clarity
class LossOutput(NamedTuple):
    l2_loss: torch.Tensor          # Reconstruction error
    l1_loss: torch.Tensor          # Sparsity penalty
    l0_loss: torch.Tensor          # Count of active features
    explained_variance: torch.Tensor # Overall variance explained
    explained_variance_A: torch.Tensor # Variance explained for Model A
    explained_variance_B: torch.Tensor # Variance explained for Model B


class CrossCoder(nn.Module):
    """
    CrossCoder model implementation based on Anthropic's paper/Neel Nanda's code.
    Encodes activations from two models into a shared latent space and decodes them back.
    """
    # ... class methods ...

    def __init__(self, cfg):
      # Initializes the CrossCoder model layers and parameters.
        super().__init__()
        self.cfg = cfg
        d_hidden = self.cfg["dict_size"]
        d_in = self.cfg["d_in"]
        n_models = 2
        self.dtype = DTYPES[self.cfg["enc_dtype"]]


        torch.manual_seed(self.cfg["seed"])


        self.W_enc = nn.Parameter(
            torch.empty(n_models, d_in, d_hidden, dtype=self.dtype)
        )
        self.b_enc = nn.Parameter(torch.zeros(d_hidden, dtype=self.dtype))


        self.W_dec = nn.Parameter(
            torch.empty(d_hidden, n_models, d_in, dtype=self.dtype)
        )
        nn.init.normal_(self.W_dec, std=1.0)


        dec_norm = self.W_dec.norm(dim=-1, keepdim=True)
        self.W_dec.data /= (dec_norm + 1e-8)
        self.W_dec.data *= self.cfg["dec_init_norm"]


        self.W_enc.data = einops.rearrange(
            self.W_dec.data.clone(),
            "d_hidden n_models d_model -> n_models d_model d_hidden",
        )


        self.b_dec = nn.Parameter(
            torch.zeros((n_models, d_in), dtype=self.dtype)
        )


        self.d_hidden = d_hidden


        self.to(self.cfg["device"])



        self.save_dir = None
        self.save_version = 0
        print(f"CrossCoder initialized on device: {self.cfg['device']}")

    def encode(self, x: torch.Tensor, apply_relu=True) -> torch.Tensor:
        # x: [batch, 2, d_model]
        # W_enc: [2, d_model, d_hidden]
        x_enc = einops.einsum(x, self.W_enc, "batch n d, n d hidden -> batch hidden")
        pre_activation = x_enc + self.b_enc
        if apply_relu:
            acts = F.relu(pre_activation)
        else:
            acts = pre_activation
        # acts: [batch, d_hidden]
        return acts

    def decode(self, acts: torch.Tensor) -> torch.Tensor:
        # acts: [batch, d_hidden]
        # W_dec: [d_hidden, 2, d_model]
        acts_dec = einops.einsum(acts, self.W_dec, "batch hidden, hidden n d -> batch n d")
        x_reconstruct = acts_dec + self.b_dec
        # x_reconstruct: [batch, 2, d_model]
        return x_reconstruct

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        acts = self.encode(x)
        x_reconstruct = self.decode(acts)
        return x_reconstruct

    def get_losses(self, x: torch.Tensor) -> LossOutput:
        # Ensure input is on the correct device and potentially cast dtype
        x = x.to(self.cfg["device"]).to(self.dtype)


        # --- Forward pass ---
        acts = self.encode(x) # Get latent activations [batch, d_hidden]
        x_reconstruct = self.decode(acts) # Get reconstructions [batch, 2,   d_model]


        # --- L2 Loss (Mean Squared Error) ---
        # Calculate squared difference between original and reconstruction
        # Use float32 for loss calculation for numerical stability
        diff = (x_reconstruct - x).float()
        squared_diff = diff.pow(2)
        # Sum squared errors over the d_model and n_models dimensions for each batch item
        l2_per_batch = einops.reduce(squared_diff, 'batch n_models d_model -> batch', 'sum')
        # Average the summed squared error across the batch
        l2_loss = l2_per_batch.mean()


        # --- L1 Loss (Sparsity Penalty) ---
        # Encourages latent activations (acts) to be zero.
        # Only non-zero activations (due to ReLU, acts >= 0) contribute.
        # We weight the penalty by the norm of the corresponding decoder weight vector.
        # This means activating features that contribute more to reconstruction (larger norm)
        # incurs a higher penalty.
        with torch.no_grad(): # Don't track gradients for norm calculation itself
             # Norm of decoder vectors (sum across models) for each latent feature
             decoder_norms = self.W_dec.norm(dim=-1) # Shape: [d_hidden, n_models]
             total_decoder_norm = einops.reduce(decoder_norms, 'd_hidden n_models -> d_hidden', 'sum')


        # Calculate L1 penalty: sum(abs(acts) * decoder_norm) averaged over batch
        # Since acts >= 0, abs(acts) = acts
        l1_loss = (acts.float() * total_decoder_norm[None, :]).sum(dim=-1).mean()


        # --- L0 Loss (Approximate Feature Count) ---
        # Counts the average number of non-zero features per input in the batch.
        l0_loss = (acts > 1e-8).float().sum(dim=-1).mean() # Use small threshold for numerical stability


        # --- Explained Variance ---
        # Measures the proportion of variance in the original data captured by the reconstruction.
        # Calculated per batch item, then averaged. Higher is better (max 1.0).
        with torch.no_grad(): # Avoid tracking gradients for variance calculation
            # Calculate total variance of the original input x (per batch item)
            # Variance = mean( (x - mean(x))^2 ) * N --> Sum( (x - mean(x))^2 )
            variance = einops.reduce((x - x.mean(dim=0, keepdim=True)).pow(2), 'batch n_models d_model -> batch', 'sum')
            # Prevent division by zero if variance is tiny
            explained_variance = (1 - l2_per_batch / (variance + 1e-8)).mean()


            # Explained variance per model (useful for seeing if one model is reconstructed better)
            variance_A = (x[:, 0] - x[:, 0].mean(dim=0, keepdim=True)).pow(2).sum(dim=-1)
            l2_per_batch_A = squared_diff[:, 0].sum(dim=-1)
            explained_variance_A = (1 - l2_per_batch_A / (variance_A + 1e-8)).mean()


            variance_B = (x[:, 1] - x[:, 1].mean(dim=0, keepdim=True)).pow(2).sum(dim=-1)
            l2_per_batch_B = squared_diff[:, 1].sum(dim=-1)
            explained_variance_B = (1 - l2_per_batch_B / (variance_B + 1e-8)).mean()




        return LossOutput(
            l2_loss=l2_loss,
            l1_loss=l1_loss,
            l0_loss=l0_loss,
            explained_variance=explained_variance,
            explained_variance_A=explained_variance_A,
            explained_variance_B=explained_variance_B
        )

    def create_save_dir(self, base_dir_str="./checkpoints"):
        base_dir = Path(base_dir_str)
        base_dir.mkdir(parents=True, exist_ok=True) # Ensure base directory exists


        # Find existing version directories
        version_list = []
        for file in base_dir.iterdir():
            if file.is_dir() and file.name.startswith("version_"):
                try:
                    version_list.append(int(file.name.split("_")[1]))
                except (IndexError, ValueError):
                    continue # Ignore directories not matching the pattern


        # Determine the next version number
        if version_list:
            version = 1 + max(version_list)
        else:
            version = 0


        self.save_dir = base_dir / f"version_{version}"
        self.save_dir.mkdir(parents=True)
        print(f"Created checkpoint directory: {self.save_dir}")

    def save(self, checkpoint_dir_str="./checkpoints"):
        """Saves the model state dictionary and config."""
        if self.save_dir is None:
            self.create_save_dir(checkpoint_dir_str)


        # Define file paths within the versioned directory
        weight_path = self.save_dir / f"crosscoder_{self.save_version}.pt"
        cfg_path = self.save_dir / f"crosscoder_{self.save_version}_cfg.json"


        # Save the model's learned parameters
        torch.save(self.state_dict(), weight_path)


        # Save the configuration used for this model
        with open(cfg_path, "w") as f:
            # Convert Path objects in config to strings for JSON serialization
            serializable_cfg = {k: str(v) if isinstance(v, Path) else v for k, v in self.cfg.items()}
            json.dump(serializable_cfg, f, indent=2)


        print(f"Saved checkpoint {self.save_version} to {self.save_dir}")
        self.save_version += 1 # Increment version for the next save

    @classmethod
    def load(cls, version_dir_str: str, checkpoint_version: int = 0):
          """Loads a CrossCoder model from a saved checkpoint directory."""
          save_dir = Path(version_dir_str)
          cfg_path = save_dir / f"crosscoder_{checkpoint_version}_cfg.json"
          weight_path = save_dir / f"crosscoder_{checkpoint_version}.pt"


          if not cfg_path.exists() or not weight_path.exists():
                raise FileNotFoundError(f"Checkpoint files not found in {save_dir} for version {checkpoint_version}")


          print(f"Loading config from: {cfg_path}")
          with open(cfg_path, "r") as f:
              cfg = json.load(f)
          print("Loaded Config:")
          pprint.pprint(cfg)


          # Create a new instance with the loaded config
          # Ensure device is handled correctly if loading on different hardware
          if 'device' not in cfg:
                cfg['device'] = 'cuda:0' if torch.cuda.is_available() else 'cpu'
                print(f"Warning: 'device' not found in config, defaulting to {cfg['device']}")
          instance = cls(cfg=cfg)


          print(f"Loading weights from: {weight_path}")
          # Load the saved weights onto the correct device specified in the config
          state_dict = torch.load(weight_path, map_location=cfg["device"])
          instance.load_state_dict(state_dict)
          print("Weights loaded successfully.")


          instance.save_dir = save_dir # Set save_dir for potential future saves
          instance.save_version = checkpoint_version + 1 # Start saving from next version


          return instance

    @classmethod
    def load_from_hf(
            cls,
            repo_id: str = "ckkissane/crosscoder-gemma-2-2b-model-diff",
            path_in_repo: str = "blocks.14.hook_resid_pre", # Original path structure
            cfg_filename: str = "cfg.json",
            weights_filename: str = "cc_weights.pt",
            device: Optional[Union[str, torch.device]] = None,
            **kwargs # Pass extra args to hf_hub_download (e.g., token)
        ):
            print(f"Loading CrossCoder from HF Hub: {repo_id}/{path_in_repo}")


            # Construct full paths within the repo
            config_repo_path = f"{path_in_repo}/{cfg_filename}"
            weights_repo_path = f"{path_in_repo}/{weights_filename}"


            # Download config and weights files
            try:
                config_path = hf_hub_download(
                    repo_id=repo_id,
                    filename=config_repo_path,
                    **kwargs
                )
                weights_path = hf_hub_download(
                    repo_id=repo_id,
                    filename=weights_repo_path,
                    **kwargs
                )
            except Exception as e:
                print(f"Error downloading from Hugging Face Hub: {e}")
                raise


            print(f"Downloaded config: {config_path}")
            print(f"Downloaded weights: {weights_path}")


            # Load config from the downloaded JSON file
            with open(config_path, 'r') as f:
                cfg = json.load(f)
            print("Loaded Config:")
            pprint.pprint(cfg)


            # Determine the device
            if device is None:
                # Use device from config if available, otherwise default
                if 'device' in cfg:
                    device = cfg['device']
                else:
                    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
                    print(f"Warning: 'device' not found in config, defaulting to {device}")
            # Update config with the determined device (important for model initialization)
            cfg["device"] = str(device)




            # Initialize CrossCoder instance with the loaded config
            instance = cls(cfg)


            # Load weights onto the specified device
            state_dict = torch.load(weights_path, map_location=device)
            instance.load_state_dict(state_dict)
            print(f"Weights loaded successfully onto device: {device}")


            return instance





Buffer

In [8]:
!pip install torch einops tqdm numpy transformer-lens


import torch # The main library for building and training neural networks (like PyTorch)
import einops # A library for easily rearranging multi-dimensional data (tensors)
import tqdm # A library to show progress bars for loops
import numpy as np # A library for numerical operations (often used alongside PyTorch)
from transformer_lens import HookedTransformer, ActivationCache # A specific library for working with Transformer language models easily

In [9]:
class Buffer:
    """
    Manages a buffer of activation pairs...
    """

    def __init__(self, cfg: dict, model_A: HookedTransformer, model_B: HookedTransformer, all_tokens: torch.Tensor):
        self.cfg = cfg
        self.device = cfg["device"]
        self.model_A = model_A
        self.model_B = model_B
        self.all_tokens = all_tokens.to(self.device)


        assert model_A.cfg.d_model == model_B.cfg.d_model
        assert model_A.cfg.d_model == cfg["d_in"]


        self.buffer_size_target = cfg["batch_size"] * cfg["buffer_mult"]
        self.buffer_batches = self.buffer_size_target // (cfg["seq_len"] - 1)
        self.buffer_size = self.buffer_batches * (cfg["seq_len"] - 1)
        print(f"Buffer target size: {self.buffer_size_target}, actual size: {self.buffer_size}")


        self.buffer = torch.zeros(
            (self.buffer_size, 2, cfg["d_in"]),
            dtype=torch.bfloat16,
            device=self.device,
            requires_grad=False,
        )
        print(f"Initialized buffer tensor with shape: {self.buffer.shape} on device: {self.device}")


        self.token_pointer = 0
        self.pointer = 0
        self.first = True
        self.normalize = True


        print("Estimating activation norm scaling factors...")
        norm_batch_size = min(cfg.get("norm_batch_size", 16), cfg["model_batch_size"])
        estimated_norm_scaling_factor_A = self.estimate_norm_scaling_factor(norm_batch_size, model_A)
        estimated_norm_scaling_factor_B = self.estimate_norm_scaling_factor(norm_batch_size, model_B)


        self.normalisation_factor = torch.tensor(
            [estimated_norm_scaling_factor_A, estimated_norm_scaling_factor_B],
            device=self.device,
            dtype=torch.float32,
        ).unsqueeze(0).unsqueeze(-1)
        print(f"Normalization factors (A, B): {self.normalisation_factor.squeeze().tolist()}")


        print("Performing initial buffer refresh...")
        self.refresh()
        print("Initial buffer refresh complete.")

    @torch.no_grad()
    def estimate_norm_scaling_factor(self, batch_size: int, model: HookedTransformer, n_batches_for_norm_estimate: int = 100) -> float:
        norms_per_batch = []
        total_estimation_tokens = batch_size * n_batches_for_norm_estimate
        if total_estimation_tokens > self.all_tokens.shape[0] * self.all_tokens.shape[1]:
            print(f"Warning: Not enough tokens ({self.all_tokens.shape[0]*self.all_tokens.shape[1]}) for full norm estimation ({total_estimation_tokens}). Using fewer batches.")
            n_batches_for_norm_estimate = (self.all_tokens.shape[0] * self.all_tokens.shape[1]) // batch_size
            if n_batches_for_norm_estimate == 0:
                print("Error: Very few tokens, cannot estimate norm factor.")
                return 1.0


        print(f"Estimating norm factor using {n_batches_for_norm_estimate} batches of size {batch_size}...")


        estimation_token_pointer = 0
        for i in tqdm.trange(n_batches_for_norm_estimate, desc=f"Estimating norm for {model.cfg.model_name}"):
            end_idx = estimation_token_pointer + batch_size
            if end_idx > self.all_tokens.shape[0]:
                print("Warning: Wrapping token dataset during norm estimation.")
                tokens = torch.cat([self.all_tokens[estimation_token_pointer:], self.all_tokens[:end_idx % self.all_tokens.shape[0]]], dim=0)
                estimation_token_pointer = end_idx % self.all_tokens.shape[0]
            else:
                tokens = self.all_tokens[estimation_token_pointer:end_idx]
                estimation_token_pointer = end_idx


            tokens = tokens.to(self.device)


            with torch.inference_mode():
                _, cache = model.run_with_cache(
                    tokens,
                    names_filter=self.cfg["hook_point"],
                    return_type=None,
                )
            acts = cache[self.cfg["hook_point"]]
            acts_no_bos = acts[:, 1:, :]
            avg_norm = acts_no_bos.norm(dim=-1, p=2).float().mean().item()
            norms_per_batch.append(avg_norm)


        mean_norm = np.mean(norms_per_batch)
        target_norm = np.sqrt(self.cfg["d_in"])
        scaling_factor = target_norm / (mean_norm + 1e-8)
        print(f"  Avg activation norm: {mean_norm:.4f}, Target norm: {target_norm:.4f}, Scaling factor: {scaling_factor:.4f}")


        return scaling_factor

    @torch.no_grad()
    def refresh(self):
        start_idx = self.buffer_size // 2 if not self.first else 0
        end_idx = self.buffer_size
        num_activations_to_generate = end_idx - start_idx


        num_sequences_needed = num_activations_to_generate // (self.cfg["seq_len"] - 1)


        print(f"Refreshing buffer: Filling indices [{start_idx}:{end_idx}] ({num_activations_to_generate} activations)")
        print(f"Need to process {num_sequences_needed} sequences.")


        write_pointer = start_idx


        model_batch_size = self.cfg["model_batch_size"]


        for i in tqdm.trange(0, num_sequences_needed, model_batch_size, desc="Refreshing Buffer"):
            batch_start = self.token_pointer
            batch_end = batch_start + model_batch_size


            if batch_end > self.all_tokens.shape[0]:
                print("Wrapping token dataset during buffer refresh.")
                tokens1 = self.all_tokens[batch_start:]
                remaining = model_batch_size - tokens1.shape[0]
                tokens2 = self.all_tokens[:remaining]
                tokens = torch.cat([tokens1, tokens2], dim=0)
                self.token_pointer = remaining
            else:
                tokens = self.all_tokens[batch_start:batch_end]
                self.token_pointer = batch_end


            tokens = tokens.to(self.device)


            with torch.inference_mode(), torch.autocast("cuda" if "cuda" in str(self.device) else "cpu", dtype=torch.bfloat16):
                _, cache_A = self.model_A.run_with_cache(
                    tokens, names_filter=self.cfg["hook_point"], return_type=None
                )
                _, cache_B = self.model_B.run_with_cache(
                    tokens, names_filter=self.cfg["hook_point"], return_type=None
                )


            acts_A = cache_A[self.cfg["hook_point"]]
            acts_B = cache_B[self.cfg["hook_point"]]


            acts_stacked = torch.stack([acts_A, acts_B], dim=0)


            acts_no_bos = acts_stacked[:, :, 1:, :]


            acts_reshaped = einops.rearrange(
                acts_no_bos,
                "n_models batch seq_len_minus_1 d_model -> (batch seq_len_minus_1) n_models d_model",
            )


            num_generated = acts_reshaped.shape[0]
            write_end = write_pointer + num_generated


            if write_end > end_idx:
                num_to_write = end_idx - write_pointer
                self.buffer[write_pointer:end_idx] = acts_reshaped[:num_to_write].to(self.buffer.dtype)
                write_pointer = end_idx
            else:
                self.buffer[write_pointer:write_end] = acts_reshaped.to(self.buffer.dtype)
                write_pointer = write_end


            if write_pointer >= end_idx:
                break


        print("Shuffling buffer...")
        permutation = torch.randperm(self.buffer_size, device=self.device)
        self.buffer = self.buffer[permutation]
        print("Buffer shuffled.")


        self.pointer = 0
        self.first = False

    @torch.no_grad() # Disable gradient calculations for this method
    def next(self) -> torch.Tensor:
        """
        Returns the next batch of activation pairs from the buffer.
        Refreshes the second half of the buffer if the pointer passes the halfway mark.


        Returns:
            torch.Tensor: A batch of normalized activation pairs,
                          shape [batch_size, 2, d_model], dtype float32.
        """
        batch_size = self.cfg["batch_size"]
        out = self.buffer[self.pointer : self.pointer + batch_size]


        # Move pointer for the next batch
        self.pointer += batch_size


        # Check if we need to refresh (pointer crossed halfway point)
        # Refresh slightly before strictly necessary to avoid waiting
        if self.pointer >= self.buffer_size // 2:
            print(f"\nBuffer pointer at {self.pointer}, refreshing...")
            self.refresh() # This refills the second half and reshuffles


        # --- Normalize and Cast ---
        # Apply pre-calculated normalization factors
        # Normalisation factor shape [1, 2, 1], out shape [batch, 2, d_model]
        if self.normalize:
            out_normalized = out.float() * self.normalisation_factor # Cast to float32 for multiplication
        else:
            out_normalized = out.float() # Still cast to float32 for training consistency


        return out_normalized






Trainer

Based on our previous tutorials, we now know how to copy and paste class methods in google colab. so from now we would just copy and paste the whole code from the appendix of the guide for better efficiency.

In [10]:
import torch
import torch.optim as optim
from torch.nn.utils import clip_grad_norm_
import wandb # Make sure wandb is installed and you're logged in via terminal (`wandb login`)
import tqdm
import math # For checking NaN/inf


class Trainer:
    """
    Orchestrates the training process for the CrossCoder model.
    Handles the training loop, optimization, learning rate scheduling,
    loss calculation, logging, and saving checkpoints.
    """
    def __init__(self, cfg: dict, crosscoder: CrossCoder, buffer: Buffer):
        """
        Initializes the Trainer.




        Args:
            cfg (dict): Configuration dictionary. Expects keys like:
                        'batch_size', 'num_tokens', 'lr', 'beta1', 'beta2',
                        'l1_coeff', 'wandb_project', 'wandb_entity', 'log_every',
                        'save_every', 'device'.
            crosscoder (CrossCoder): The CrossCoder model instance to train.
            buffer (Buffer): The initialized Buffer instance providing activation data.
        """
        self.cfg = cfg
        self.crosscoder = crosscoder
        self.buffer = buffer
        self.device = cfg["device"]




        # Calculate total training steps based on tokens and batch size
        self.total_steps = cfg["num_tokens"] // cfg["batch_size"]
        print(f"Total training steps calculated: {self.total_steps}")




        # --- Optimizer ---
        # Adam optimizer is commonly used for training neural networks
        self.optimizer = optim.Adam(
            self.crosscoder.parameters(), # Pass the model's learnable parameters
            lr=cfg["lr"],                 # Learning rate
            betas=(cfg["beta1"], cfg["beta2"]), # Adam specific hyperparameters
            # weight_decay=cfg.get("weight_decay", 0.0) # Optional: L2 regularization on weights
        )
        print(f"Initialized Adam optimizer with LR: {cfg['lr']}")




        # --- Learning Rate Scheduler ---
        # Defines how the learning rate changes during training.
        # Here: constant LR for 80% of steps, then linear decay to 0.
        self.scheduler = optim.lr_scheduler.LambdaLR(
            self.optimizer, self.lr_lambda # lr_lambda is a method defined below
        )
        print("Initialized LR scheduler (constant then linear decay).")




        # --- Step Counter ---
        self.step_counter = 0




        # --- WandB Initialization ---
        # Connect to Weights & Biases for logging
        try:
            wandb.init(
                project=cfg["wandb_project"],
                entity=cfg["wandb_entity"],
                config=cfg, # Log the configuration used for this run
                name=cfg.get("wandb_run_name", None) # Optional: custom run name
            )
            print(f"WandB initialized for project '{cfg['wandb_project']}' entity '{cfg['wandb_entity']}'.")
            # Watch the model gradients and parameters (optional, can be verbose)
            # wandb.watch(self.crosscoder, log_freq=max(100, cfg['log_every']), log='all')
        except Exception as e:
            print(f"Error initializing WandB: {e}")
            print("WandB logging will be disabled.")
            # Optionally disable wandb if init fails
            # wandb.init(mode="disabled")


    def lr_lambda(self, step: int) -> float:
        """
        Learning rate schedule: Constant for 80% of training, then linear decay.


        Args:
            step (int): The current training step number.


        Returns:
            float: The learning rate multiplier for the current step.
        """
        warmup_steps = self.cfg.get("lr_warmup_steps", 0) # Optional warmup
        decay_start_step = int(self.cfg.get("lr_decay_start_fraction", 0.8) * self.total_steps)


        if step < warmup_steps:
            # Linear warmup from 0 to 1
            return float(step) / float(max(1, warmup_steps))
        elif step < decay_start_step:
            # Constant phase
            return 1.0
        else:
            # Linear decay phase
            steps_into_decay = step - decay_start_step
            total_decay_steps = self.total_steps - decay_start_step
            decay_factor = 1.0 - (steps_into_decay / total_decay_steps)
            # Ensure decay factor doesn't go below a minimum (e.g., 0)
            return max(0.0, decay_factor)


    def get_l1_coeff(self) -> float:
        """
        L1 coefficient schedule: Linear ramp-up, then constant.
        Helps the model focus on reconstruction first, then sparsity.


        Returns:
            float: The L1 coefficient for the current step.
        """
        l1_warmup_frac = self.cfg.get("l1_warmup_fraction", 0.05) # Fraction of total steps for ramp-up
        l1_warmup_steps = int(l1_warmup_frac * self.total_steps)
        target_l1_coeff = self.cfg["l1_coeff"]


        if self.step_counter < l1_warmup_steps:
            # Linear ramp-up from 0 to target_l1_coeff
            return target_l1_coeff * (self.step_counter / max(1, l1_warmup_steps))
        else:
            # Constant phase at the target coefficient
            return target_l1_coeff




    def step(self) -> dict:
        """
        Performs a single training step:
        1. Get data batch from buffer.
        2. Run forward pass and calculate losses.
        3. Run backward pass to get gradients.
        4. Clip gradients.
        5. Update model weights using optimizer.
        6. Update learning rate using scheduler.
        7. Zero gradients.
        8. Return loss dictionary for logging.


        Returns:
            dict: Dictionary containing loss values and other metrics for this step.
        """
        # Set model to training mode (important for dropout, batchnorm layers, though not used here)
        self.crosscoder.train()


        # 1. Get data batch
        # Buffer's next() method handles normalization
        acts = self.buffer.next() # Shape: [batch_size, 2, d_model], dtype float32


        # 2. Forward pass and loss calculation
        losses: LossOutput = self.crosscoder.get_losses(acts)
        current_l1_coeff = self.get_l1_coeff()
        loss = losses.l2_loss + current_l1_coeff * losses.l1_loss


        # Check for NaN/inf loss, which indicates instability
        if not torch.isfinite(loss):
              print(f"\nWarning: Non-finite loss detected at step {self.step_counter}: {loss.item()}")
              print(f"  L2: {losses.l2_loss.item()}, L1: {losses.l1_loss.item()}, L1 Coeff: {current_l1_coeff}")
              # Optional: Add more debugging here, e.g., check gradients later
              # For now, just return the problematic losses without backprop
              loss_dict = {
                  "loss": loss.item(), # Log the NaN/inf
                  "l2_loss": losses.l2_loss.item(),
                  "l1_loss": losses.l1_loss.item(),
                  "l0_loss": losses.l0_loss.item(),
                  "l1_coeff": current_l1_coeff,
                  "lr": self.scheduler.get_last_lr()[0],
                  "explained_variance": losses.explained_variance.item(),
                  "explained_variance_A": losses.explained_variance_A.item(),
                  "explained_variance_B": losses.explained_variance_B.item(),
              }
              self.step_counter += 1 # Still increment step counter
              return loss_dict # Skip backprop and optimizer step




        # 3. Backward pass (calculate gradients)
        # Gradients are calculated for the combined loss
        loss.backward()


        # 4. Clip gradients (prevent exploding gradients)
        # max_norm=1.0 is a common value
        grad_norm = clip_grad_norm_(self.crosscoder.parameters(), max_norm=1.0)


        # 5. Optimizer step (update weights)
        self.optimizer.step()


        # 6. Scheduler step (update learning rate)
        self.scheduler.step()


        # 7. Zero gradients (clear gradients for the next iteration)
        self.optimizer.zero_grad()


        # 8. Prepare loss dictionary for logging
        loss_dict = {
            "loss": loss.item(),
            "l2_loss": losses.l2_loss.item(),
            "l1_loss": losses.l1_loss.item(),
            "l0_loss": losses.l0_loss.item(),
            "l1_coeff": current_l1_coeff,
            "lr": self.scheduler.get_last_lr()[0],
            "explained_variance": losses.explained_variance.item(),
            "explained_variance_A": losses.explained_variance_A.item(),
            "explained_variance_B": losses.explained_variance_B.item(),
            "grad_norm": grad_norm.item() # Log gradient norm after clipping
        }


        self.step_counter += 1
        return loss_dict


    def log(self, loss_dict: dict):
        """Logs the metrics dictionary to WandB and prints it."""
        # Log to WandB if initialized
        if wandb.run is not None:
            wandb.log(loss_dict, step=self.step_counter)


        # Print metrics locally (optional, can be verbose)
        log_str = f"Step: {self.step_counter:>7} | "
        log_str += f"Loss: {loss_dict['loss']:.4f} | "
        log_str += f"L2: {loss_dict['l2_loss']:.4f} | "
        log_str += f"L1: {loss_dict['l1_loss']:.4f} | "
        log_str += f"L0: {loss_dict['l0_loss']:.2f} | "
        log_str += f"ExplVar: {loss_dict['explained_variance']:.3f} | "
        log_str += f"LR: {loss_dict['lr']:.2e} | "
        log_str += f"L1Coeff: {loss_dict['l1_coeff']:.2f}"
        # Avoid printing every single step if logging frequently
        if self.step_counter % (self.cfg['log_every'] * 10) == 0 or self.step_counter <= 10:
              print(log_str)


    def save(self):
        """Saves a checkpoint of the CrossCoder model."""
        print(f"\nSaving checkpoint at step {self.step_counter}...")
        self.crosscoder.save(checkpoint_dir_str=self.cfg.get("checkpoint_path", "./checkpoints"))


    def train(self):
        """Runs the main training loop."""
        print("\n--- Starting Training ---")
        self.step_counter = 0 # Reset step counter at the beginning of training


        try:
            # Use tqdm for a progress bar over the total steps
            for i in tqdm.trange(self.total_steps, desc="Training Progress"):
                # Perform one training step
                loss_dict = self.step()


                # Log metrics periodically
                if i % self.cfg["log_every"] == 0:
                    self.log(loss_dict)


                # Save checkpoint periodically
                # Save at i+1 so the checkpoint name reflects the step *completed*
                if (i + 1) % self.cfg["save_every"] == 0:
                    self.save()


                # Optional: Add early stopping logic here if desired


        except KeyboardInterrupt:
            print("\nTraining interrupted by user.")
        except Exception as e:
              print(f"\nAn error occurred during training: {e}")
              import traceback
              traceback.print_exc() # Print detailed traceback
        finally:
            # --- Final Save ---
            # Always save the final model state, regardless of how training ended
            print("\n--- Training Finished or Interrupted ---")
            self.save()
            if wandb.run is not None:
                wandb.finish() # Ensure WandB run is marked as finished
                print("WandB run finished.")

In [11]:
!pip install torch transformer-lens wandb numpy

Training Script

In [12]:
import torch
from transformer_lens import HookedTransformer
import os # Used for operating system interactions (like checking paths, though pathlib is used here)
import wandb # Weights & Biases library for experiment tracking
from pathlib import Path # Modern way to handle file paths
import numpy as np

In [13]:
# --- 1. Configuration ---
# Adjust paths and WandB details as needed!
cfg = {
    # --- Setup ---
    "seed": 49,
    "device": "cuda:0" if torch.cuda.is_available() else "cpu",
    "wandb_project": "crosscoder", # <<< CHANGE THIS
    "wandb_entity": "your-wandb-username",  # <<< CHANGE THIS
    "wandb_run_name": None,
    "checkpoint_path": "./checkpoints/",
    "data_cache_dir": "./cache/",
    "data_dir": "./data/",


    # --- Model Config ---
    "model_name_A": "google/gemma-2-2b",
    "model_name_B": "google/gemma-2-2b-it",
    "hook_point": "blocks.14.hook_resid_pre",
    # "d_in" will be set automatically


    # --- CrossCoder/SAE Config ---
    "dict_size": 2**14, # Latent dimension (16384)
    "enc_dtype": "fp32", # CrossCoder weight data type
    "dec_init_norm": 0.08, # Initial decoder weight norm


    # --- Training Hyperparameters ---
    "lr": 5e-5, # Learning rate
    "beta1": 0.9, # Adam optimizer beta1
    "beta2": 0.999, # Adam optimizer beta2
    "l1_coeff": 2.0, # Sparsity penalty strength
    "lr_warmup_steps": 0, # Learning rate warmup
    "lr_decay_start_fraction": 0.8, # When to start LR decay
    "l1_warmup_fraction": 0.05, # When L1 penalty reaches full strength


    # --- Data & Batching ---
    "num_tokens": 10_000_000, # Total tokens for training
    "batch_size": 4096,      # CrossCoder training batch size
    "seq_len": 1024,         # LLM sequence length
    "model_batch_size": 4,   # LLM inference batch size (for buffer refresh)
    "buffer_mult": 128,      # Buffer size multiplier
    "norm_batch_size": 16,   # Batch size for norm estimation


    # --- Logging & Saving ---
    "log_every": 100,        # Log to WandB frequency
    "save_every": 20000,     # Checkpoint saving frequency
}


# ... (Optional Argument Parsing) ...


# --- Set Seed ---
torch.manual_seed(cfg["seed"])
np.random.seed(cfg["seed"]) # If using numpy random functions


# --- Create Directories ---
Path(cfg["checkpoint_path"]).mkdir(parents=True, exist_ok=True)
Path(cfg["data_cache_dir"]).mkdir(parents=True, exist_ok=True)
Path(cfg["data_dir"]).mkdir(parents=True, exist_ok=True)


# --- 2. Load Models ---
print(f"Loading models onto device: {cfg['device']}")
model_A = HookedTransformer.from_pretrained(
    cfg["model_name_A"],
    device=cfg["device"],
    torch_dtype=torch.bfloat16, # Load weights in bfloat16
    # token=YOUR_HF_TOKEN # Add if needed
)
model_B = HookedTransformer.from_pretrained(
    cfg["model_name_B"],
    device=cfg["device"],
    torch_dtype=torch.bfloat16,
    # token=YOUR_HF_TOKEN # Add if needed
)


model_A.eval()
model_B.eval()


cfg["d_in"] = model_A.cfg.d_model # Get activation dimension
print(f"Model hidden dimension (d_in): {cfg['d_in']}")


# --- 3. Load Token Data ---
print("Loading token data...")
all_tokens = load_pile_lmsys_mixed_tokens(
    cache_dir=cfg["data_cache_dir"],
    data_dir=cfg["data_dir"]
)
print(f"Loaded {all_tokens.shape[0]} sequences of length {all_tokens.shape[1]}.")

  # --- 4. Instantiate CrossCoder ---
print("Instantiating CrossCoder...")
crosscoder = CrossCoder(cfg)


# --- 5. Instantiate Buffer ---
print("Instantiating Buffer (will estimate norms and refresh)...")
buffer = Buffer(cfg, model_A, model_B, all_tokens)

# --- 6. Instantiate Trainer ---
print("Instantiating Trainer...")
trainer = Trainer(cfg, crosscoder, buffer)

# --- 7. Start Training ---
print("\nStarting training run...")
trainer.train()
print("\n--- Training script finished ---")


Loading models onto device: cuda:0


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

...

Shuffling buffer...
Buffer shuffled.


Training Progress: 100%|██████████| 2441/2441 [23:38<00:00,  1.72it/s]



--- Training Finished or Interrupted ---

Saving checkpoint at step 2441...
Created checkpoint directory: checkpoints/version_0
Saved checkpoint 0 to checkpoints/version_0


explained_variance,▁▆▇▇█████████████████████
explained_variance_A,▁▆▇▇█████████████████████
explained_variance_B,▁▆▇▇█████████████████████
grad_norm,█▄▃▃▄▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁
l0_loss,██▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
l1_coeff,▁▇███████████████████████
l1_loss,▁█▆▆▆▆▅▅▅▅▅▅▅▅▅▄▄▄▄▄▄▄▄▄▄
l2_loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,████████████████████▇▆▄▃▁
explained_variance,0.69202


WandB run finished.

--- Training script finished ---


Analysis of the Result

In [14]:
# Cell: Analyze Trained CrossCoder (Run AFTER training finishes)


import torch
import plotly.express as px
from pathlib import Path
import numpy as np


# Ensure the CrossCoder class definition cell has been run


# --- 1. Load your locally trained CrossCoder ---
# Set the path to your checkpoint directory (should match cfg['checkpoint_path'])
base_checkpoint_dir = Path("./checkpoints/")


# Find the latest version directory (e.g., version_0, version_1, etc.)
version_dirs = sorted([d for d in base_checkpoint_dir.iterdir() if d.is_dir() and d.name.startswith("version_")])
if not version_dirs:
    print(f"Error: No version directories found in {base_checkpoint_dir}")
    # Handle error appropriately, maybe raise exception or exit
    raise FileNotFoundError(f"No version directories found in {base_checkpoint_dir}")
else:
    latest_version_dir = version_dirs[-1] # Get the latest version directory path
    print(f"Using latest checkpoint directory: {latest_version_dir}")


    # Find the latest checkpoint number within that directory
    checkpoint_files = list(latest_version_dir.glob("crosscoder_*.pt"))
    if not checkpoint_files:
        print(f"Error: No checkpoint files (.pt) found in {latest_version_dir}")
        raise FileNotFoundError(f"No checkpoint files (.pt) found in {latest_version_dir}")
    else:
        # Extract version numbers from filenames and find the latest
        versions = []
        for f in checkpoint_files:
             try:
                 versions.append(int(f.stem.split('_')[1]))
             except (IndexError, ValueError):
                 continue # Ignore files not matching the pattern
        if not versions:
             print(f"Error: Could not parse checkpoint version numbers in {latest_version_dir}")
             raise ValueError(f"Could not parse checkpoint version numbers in {latest_version_dir}")


        latest_checkpoint_version = max(versions)
        print(f"Loading latest checkpoint: version {latest_checkpoint_version}")


        # Load the CrossCoder using the .load() method
        try:
            # Disable gradient calculations for analysis
            torch.set_grad_enabled(False)
            cross_coder = CrossCoder.load(str(latest_version_dir), latest_checkpoint_version)
            cross_coder.eval() # Set model to evaluation mode
            print("Trained CrossCoder loaded successfully.")


            # --- 2. Perform Analysis (Calculations) ---
            print("Analyzing decoder weights...")
            W_dec = cross_coder.W_dec.data.float() # Use float32 for analysis stability


            # Calculate norms for each model's decoder vectors per feature
            norms = W_dec.norm(dim=-1, p=2) # Shape: [d_hidden, 2]
            norms_A = norms[:, 0]
            norms_B = norms[:, 1]


            # Calculate relative norm contribution of Model B
            total_norm = norms.sum(dim=-1)
            relative_norms_B = norms_B / (total_norm + 1e-8) # Shape: [d_hidden]


            # Identify shared features based on relative norms
            shared_latent_mask = (relative_norms_B > 0.3) & (relative_norms_B < 0.7) # Shape: [d_hidden]
            num_shared = shared_latent_mask.sum().item()
            print(f"Found {num_shared} shared features (relative norm between 0.3 and 0.7).")


            # Calculate cosine similarity for shared features
            W_dec_A_shared = W_dec[shared_latent_mask, 0, :]
            W_dec_B_shared = W_dec[shared_latent_mask, 1, :]


            dot_products = (W_dec_A_shared * W_dec_B_shared).sum(dim=-1)
            norms_product = W_dec_A_shared.norm(dim=-1, p=2) * W_dec_B_shared.norm(dim=-1, p=2)
            cosine_sims = dot_products / (norms_product + 1e-8) # Shape: [num_shared]


            # --- 3. Plotting Results ---
            print("Generating plots...")


            # Plot 1: Histogram of Relative Decoder Norms
            fig_rel_norms = px.histogram(
                relative_norms_B.cpu().numpy(),
                title=f"Relative Decoder Norm ({cross_coder.cfg.get('model_name_B', 'Model B')} / Total) - Layer {cross_coder.cfg.get('hook_point', 'N/A')}",
                labels={"value": "Relative Decoder Norm Strength (Model B)"},
                nbins=200,
            )
            fig_rel_norms.update_layout(showlegend=False)
            fig_rel_norms.update_yaxes(title_text="Number of Latent Features")
            fig_rel_norms.update_xaxes(
                tickvals=[0, 0.25, 0.5, 0.75, 1.0],
                ticktext=['0 (Model A)', '0.25', '0.5 (Shared)', '0.75', '1.0 (Model B)']
            )
            fig_rel_norms.show()


            # Plot 2: Histogram of Cosine Similarities for Shared Features
            if num_shared > 0:
                fig_cos_sim = px.histogram(
                    cosine_sims.cpu().numpy(),
                    title=f"Cosine Similarity of Decoder Vectors for Shared Features - Layer {cross_coder.cfg.get('hook_point', 'N/A')}",
                    log_y=True,  # Sets the y-axis to log scale
                    range_x=[-1, 1],  # Sets the x-axis range from -1 to 1
                    nbins=100,
                    labels={"value": "Cosine Similarity"}
                )
                fig_cos_sim.update_layout(showlegend=False)
                fig_cos_sim.update_yaxes(title_text="Number of Shared Latents (log scale)")
                fig_cos_sim.show()
            else:
                print("Skipping cosine similarity plot as no shared features were found based on the criteria.")


        except FileNotFoundError as e:
            print(e)
        except Exception as e:
            print(f"An error occurred during loading or analysis: {e}")
            import traceback
            traceback.print_exc()
        finally:
            # Re-enable gradients if needed for subsequent cells
            torch.set_grad_enabled(True)


Using latest checkpoint directory: checkpoints/version_0
Loading latest checkpoint: version 0
Loading config from: checkpoints/version_0/crosscoder_0_cfg.json
Loaded Config:
{'batch_size': 4096,
 'beta1': 0.9,
 'beta2': 0.999,
 'buffer_mult': 128,
 'checkpoint_path': './checkpoints/',
 'd_in': 2304,
 'data_cache_dir': './cache/',
 'data_dir': './data/',
 'dec_init_norm': 0.08,
 'device': 'cuda:0',
 'dict_size': 16384,
 'enc_dtype': 'fp32',
 'hook_point': 'blocks.14.hook_resid_pre',
 'l1_coeff': 2.0,
 'l1_warmup_fraction': 0.05,
 'log_every': 100,
 'lr': 5e-05,
 'lr_decay_start_fraction': 0.8,
 'lr_warmup_steps': 0,
 'model_batch_size': 4,
 'model_name_A': 'google/gemma-2-2b',
 'model_name_B': 'google/gemma-2-2b-it',
 'norm_batch_size': 16,
 'num_tokens': 10000000,
 'save_every': 20000,
 'seed': 49,
 'seq_len': 1024,
 'wandb_entity': 'kasmik-backups-kathmandu-university',
 'wandb_project': 'crosscoder',
 'wandb_run_name': None}
CrossCoder initialized on device: cuda:0
Loading weights fr